Lets create a simple Tensor-likes "Tensors". So we are just gonna think of tensors, but in the end, they are a bit different to this. However, it has the same bases.\
A vector are $x_i$ values in $R^1$, a matrix are $x_i$ values in $R^2$.\
And the new: The tensor is $R^i$ higher than $i > 2$.

In [4]:
from typing import List

In [2]:
Tensor = list

In [5]:
def shape(tensor: Tensor) -> List[int]:
    sizes: List[int] = []
    while isinstance(tensor, list):
        sizes.append(len(tensor))
        tensor = tensor[0]
    return sizes

In [6]:
shape([1,2,3])

[3]

If the tensor is $R^1$ then is a vector, however if the tensor has a list or vector or even a matrix in its index, then its a higher-order tensor.\
Lets create a function to know this.

In [13]:
def is_1d(tensor: Tensor) -> bool:
    return not isinstance(tensor[0], list)

In [17]:
#Tensor summatory
def tensor_sum(tensor: Tensor) -> float:
    if is_1d(tensor):
        return sum(tensor) # We can use ordinary Python sum
    else:
        return sum(tensor_sum(tensor_i) for tensor_i in tensor) # We use itirable sum

In [16]:
from typing import Callable
def tensor_apply(f: Callable[[float], float], tensor: Tensor) -> Tensor:
    if is_1d(tensor):
        return [f(x) for x in tensor]
    else:
        return [tensor_apply(f, tensor_i) for tensor_i in tensor]

In [12]:
assert tensor_apply(lambda x: x + 1, [1, 2, 3]) == [2, 3, 4]
assert tensor_apply(lambda x: 2 * x, [[1, 2], [3, 4]]) == [[2, 4], [6, 8]]

In [18]:
def zeros_like(tensor: Tensor) -> Tensor:
    return tensor_apply(lambda _: 0.0, tensor)

In [20]:
def tensor_combine(f: Callable[[float, float], float], t1: Tensor, t2: Tensor) -> Tensor:
    if is_1d(t1):
        return [f(x, y) for x, y in zip(t1, t2)]
    else:
        return [tensor_combine(f, t1_i, t2_i) for t1_i, t2_i in zip(t1, t2)]

Ok, we just created the basic functions to start working and creating better NeuralNetworks. We added summatory, component sum, and a generalized method for functions. 

In [22]:
from typing import Iterable, Tuple
class Layer:
    """
    Our neural networks will be composed of Layers, each of which
    knows how to do some computation on its inputs in the "forward"
    direction and propagate gradients in the "backward" direction.
    """
    def forward(self, input):
        """
        Note the lack of types. We're not going to be prescriptive
        about what kinds of inputs layers can take and what kinds
        of outputs they can return.
        """
        raise NotImplementedError
    def backward(self, gradient):
        """
        Similarly, we're not going to be prescriptive about what the
        gradient looks like. It's up to you the user to make sure
        that you're doing things sensibly.
        """
        raise NotImplementedError
    def params(self) -> Iterable[Tensor]:
        """
        Returns the parameters of this layer. The default implementation
        returns nothing, so that if you have a layer with no parameters
        you don't have to implement this.
        """
        return ()
    def grads(self) -> Iterable[Tensor]:
        """
        Returns the gradients, in the same order as params().
        """
        return ()

In [24]:
from  models import sigmoid
class Sigmoid(Layer):
    def forward(self, input: Tensor) -> Tensor:
        """
        Apply sigmoid to each element of the input tensor,
        and save the results to use in backpropagation.
        """
        self.sigmoids = tensor_apply(sigmoid, input)
        return self.sigmoids
    def backward(self, gradient: Tensor) -> Tensor:
        return tensor_combine(lambda sig, grad: sig * (1 - sig) * grad, self.sigmoids, gradient)

ModuleNotFoundError: No module named 'models'